In [11]:
%load_ext autoreload
%autoreload 2

import os

os.chdir('/data/core-rad/tobweber/bernoulli-mri')

from typing import Tuple
import torch
from torch import Tensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from src.utils import ifft2c, get_top_k_mask
from src.datasets import KneeDataset
from src.evaluation import MetricAgent
from tqdm import tqdm
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Initialize Metric Agent

In [2]:
metric_agent = MetricAgent()

result = {
    'dataset': [],
    'method': [],
    'acc_fac': [],
    'psnr': [],
    'ssim': [],
    'nmse': []
}

In [3]:
def get_metrics(dl: DataLoader, mask: Tensor) -> Tuple[float, float, float]:
    for batch in tqdm(dl, leave=False):
        img_k = batch['k_space'].cuda()
        img = batch['img'].cuda()

        img_pred = ifft2c(img_k * mask + 0.0)
        img_mag = torch.abs(img_pred)

        metric_agent(img_mag, img)

    m = metric_agent.aggregate()

    return m['PSNR'], m['SSIM'], m['NMSE']

# KNEE DATASET

In [4]:
ds = KneeDataset('/data/core-rad/data/knee_fastmri', train=False)
dl = DataLoader(ds, batch_size=64, num_workers=24)

## Equispaced

In [5]:
from src.mask_patterns import EquiSpacedMaskFunc

num_runs = 10
acc_facs = [4, 8, 16, 32]

for _ in range(num_runs):
    for a in acc_facs:
        mask = EquiSpacedMaskFunc([0.04], [a])((1, 320, 320))[0].cuda()
        mask = mask.squeeze(2).expand(320, -1)
        mask = mask.unsqueeze(0).unsqueeze(0)

        psnr, ssim, nmse = get_metrics(dl, mask)

        result['dataset'].append('knee')
        result['method'].append('equi')
        result['acc_fac'].append(a)
        result['psnr'].append(psnr)
        result['ssim'].append(ssim)
        result['nmse'].append(nmse)


## 2D Variable Density

In [6]:
from src.mask_patterns import get_2d_variable_density_mask

num_runs = 10
acc_facs = [4, 8, 16, 32]

for _ in range(num_runs):
    for a in acc_facs:
        mask = get_2d_variable_density_mask(320, a, 30).cuda()
        mask = mask.unsqueeze(0).unsqueeze(0)

        psnr, ssim, nmse = get_metrics(dl, mask)

        result['dataset'].append('knee')
        result['method'].append('var_dens')
        result['acc_fac'].append(a)
        result['psnr'].append(psnr)
        result['ssim'].append(ssim)
        result['nmse'].append(nmse)

## PROM

In [7]:
num_runs = 10
acc_facs = [4, 8, 16, 32]

for run_idx in range(1, num_runs + 1):
    for a in acc_facs:
        path = f'logs/knee_2d_a{a}/knee_r{run_idx}.pt'
        score = torch.load(path)['scores'][-1].cuda()

        mask = get_top_k_mask(score.squeeze(), a)

        psnr, ssim, nmse = get_metrics(dl, mask)

        result['dataset'].append('knee')
        result['method'].append('prom')
        result['acc_fac'].append(a)
        result['psnr'].append(psnr)
        result['ssim'].append(ssim)
        result['nmse'].append(nmse)

## IGS Sampling

In [9]:
from src.igs import IGS

num_runs = 10
acc_facs = [4, 8, 16, 32]

for run_idx in range(1, num_runs + 1):
    masks = torch.load(f'logs/IGS/igs_knee_{run_idx}.pt').cuda()
    ns = [IGS.get_n(acc_fac=a, img_size=320) for a in acc_facs]

    for n, a in zip(ns, acc_facs):
        mask = masks[n - 2].unsqueeze(0)
        mask = mask.expand(320, -1)
        mask = mask.unsqueeze(0).unsqueeze(0)

        psnr, ssim, nmse = get_metrics(dl, mask)

        result['dataset'].append('knee')
        result['method'].append('igs')
        result['acc_fac'].append(a)
        result['psnr'].append(psnr)
        result['ssim'].append(ssim)
        result['nmse'].append(nmse)

In [12]:
df = pd.DataFrame.from_dict(result)

df.head()

df.to_csv('logs/knee.csv')

In [13]:
for group_name, df_group in df.groupby(['method', 'acc_fac']):
        
    print('GROUP:', group_name)
    print('PSNR MEAN {:.3f} | SD {:.3f}'.format(df_group['psnr'].mean(), df_group['psnr'].std()))
    print('SSIM MEAN {:.3f} | SD {:.3f}'.format(df_group['ssim'].mean(), df_group['ssim'].std()))
    print('NMSE MEAN {:.3f} | SD {:.3f}'.format(df_group['nmse'].mean(), df_group['nmse'].std()))
    print('------------------------------------')

GROUP: ('equi', 4)
PSNR MEAN 24.481 | SD 0.518
SSIM MEAN 0.601 | SD 0.030
NMSE MEAN 0.066 | SD 0.008
------------------------------------
GROUP: ('equi', 8)
PSNR MEAN 23.803 | SD 0.226
SSIM MEAN 0.524 | SD 0.012
NMSE MEAN 0.077 | SD 0.004
------------------------------------
GROUP: ('equi', 16)
PSNR MEAN 23.365 | SD 0.130
SSIM MEAN 0.474 | SD 0.004
NMSE MEAN 0.085 | SD 0.002
------------------------------------
GROUP: ('equi', 32)
PSNR MEAN 23.158 | SD 0.126
SSIM MEAN 0.448 | SD 0.004
NMSE MEAN 0.090 | SD 0.002
------------------------------------
GROUP: ('igs', 4)
PSNR MEAN 28.553 | SD 0.000
SSIM MEAN 0.645 | SD 0.000
NMSE MEAN 0.031 | SD 0.000
------------------------------------
GROUP: ('igs', 8)
PSNR MEAN 26.428 | SD 0.000
SSIM MEAN 0.532 | SD 0.000
NMSE MEAN 0.047 | SD 0.000
------------------------------------
GROUP: ('igs', 16)
PSNR MEAN 24.376 | SD 0.000
SSIM MEAN 0.458 | SD 0.000
NMSE MEAN 0.070 | SD 0.000
------------------------------------
GROUP: ('igs', 32)
PSNR MEAN 22.31